 # An Example of using `minimod`
 For this notebook, we will first create the data needs to be used for the program. Then we will instantiate the class and make a report of the results using the `report` method

In [1]:
import sys
import pandas as pd

import minimod as mm



Using Python-MIP package version 1.6.8


 ## Data
 This is how the data was processed, assuming we use the Katie_VA_Benefits_and_Costs_1_8_2019.xlsx file.
 ```python
 excel_file = "/home/lordflaron/Documents/GAMS-Python/Cameroon VA/GAMS_Working/GAMS_R Project/Katie_VA_Benefits_and_Costs_1_8_2019.xlsx"
 df_benefit = (pd.read_excel(excel_file,
                            sheet_name = 'Benefits',
                            header = 2)
               .set_index(['intervention', 'space'])
               .stack()
               .to_frame()
               .reset_index()
               .rename({'level_2' : 'time',
                        0 : 'benefit'}, axis=1)
               .set_index(['intervention', 'space', 'time'])
               )
 df_cost = (pd.read_excel(excel_file,
                            sheet_name = 'Costs',
                            header = 2)
               .set_index(['intervention', 'space'])
               .stack()
               .to_frame()
               .reset_index()
               .rename({'level_2' : 'time',
                        0 : 'costs'}, axis=1)
               .set_index(['intervention', 'space', 'time'])
               )
 ```

 Then we merge the cost and benefit data together.
 ```python
 df = (df_benefit
       .merge(df_cost, left_index=True, right_index=True)
 )
```
 Then we save the data. The finished data can be found in the `/examples/data` folder.
 `df.to_csv("./data/example1.csv")`

In [2]:

df = pd.read_csv("data/example1.csv")

 ## Running the model
 Now we instantiate the model, and then run `fit` and get the report.

In [3]:
c = mm.CostSolver(data = df)


In [4]:

opt = c.fit()



              Loading MIP Model with:
              Solver = CBC
              Method = min
              
[Note]: Optimal Solution Found


In [5]:

c.report()




AttributeError: 'CostSolver' object has no attribute '_opt_df'